# Data Prep
Our dataset already has an appropraite file structure with class divided within each of the folder (train/test).  
In this document, we will just create a validation set.

In [5]:
import os, shutil
import numpy as np
import tensorflow as tf

In [3]:
train_dir = 'DATA/train'
test_dir = 'DATA/test'
val_dir = 'DATA/validation'

### Check for Corrupted Images
(modified the original code from Keras.io)

In [7]:
for dirp in (f'{train_dir}/NORMAL', f'{train_dir}/PNEUMONIA', f'{test_dir}/NORMAL', f'{test_dir}/PNEUMONIA'):
    for fn in os.listdir(dirp):
        fp = os.path.join(dirp, fn)
        try:
            fobj = open(fp, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            print(f'{fp} appears corrupted')
            # os.remove(fp) # remo

DATA/train/PNEUMONIA\BACTERIA-37006-0002.jpeg appears corrupted


### Subsetting Validation Set

In [4]:
# make necessary directories
def makedirect(paths):
    for path in paths:
        try: os.mkdir(path)
        except FileExistsError: 
            print(f'{path} already exists')

makedirect([val_dir, f'{val_dir}/NORMAL', f{val_dir}/PNEUMONIA'])

In [11]:
# pull images from training into validation sets
normal_imgs = [fn for fn in os.listdir(f'{train_dir}/NORMAL') if fn.endswith('.jpeg')]
pneumo_imgs = [fn for fn in os.listdir(f'{train_dir}/PNEUMONIA') if fn.endswith('.jpeg')]

# randomly select 20% of it 
normal_val = np.random.choice(normal_imgs, int(.2 * len(normal_imgs)), replace = False)
pneumo_val = np.random.choice(pneumo_imgs, int(.2 * len(pneumo_imgs)), replace = False)   


In [13]:
# move files
def move_files(olddir, newdir, fnames):
    for fn in fnames:
        shutil.move(f'{olddir}/{fn}', f'{newdir}/{fn}')

move_files(f'{train_dir}/NORMAL', f'{val_dir}/NORMAL', normal_val)
move_files(f'{train_dir}/PNEUMONIA', f'{val_dir}/PNEUMONIA', pneumo_val)